<div>
<table style="width:100%; background-color:transparent;">
  <tr style="background-color:transparent;">
    <td align="left"; style="background-color:transparent; width: 50%;"><a href="https://www.inrae.fr"><img src="https://www.inrae.fr/themes/custom/inrae_socle/logo.svg" width="30%"></a></td>
    <td align="right"; style="background-color:transparent; width: 50%;"><a href = "https://www.vet-alfort.fr/"><img src="https://www.vet-alfort.fr/images/logo-enva.svg" width="30%"></a></td>
  </tr>
</table> 
</div>

<center><h1>Bovine Embryos Movies challenge</h1></center>

<center><h3>A data challenge on early prediction of the fate of bovine embryos</h3></center>
<br/>
<center><i>Julien Chiquet (MIA Paris-Saclay, Inrae), Pierre Gloaguen (MIA Paris-Saclay, AgroParisTech), Nicolas Jouvin (MIA Paris-Saclay), Patrick Bouthemy (SERPICO, Inria), Alain Truibil (MaiAGE, Inrae), Alline Reis (PASP, ENVA) </i></center>

# 1. Introduction 


## Objective of the challenge

This challenge consists in predicting the state of bovine embryos as early as possible after in-vitro fertilization.

We have defined 8 different classes (denoted from "A" to "H" in this challenge) corresponding to 8 different biological states from alive ("A") to dead ("H"). These labels express the state of the embryo after 8 days. However, it is of great interest to be able to predict this future state earlier. **The goal of this challenge is to make this prediction between 1 and 4 days.**

For this, you have access to a set of videos (timelapses) of bovine embryo development for a total of 300 snapshots taken every fifteen minutes.

> *Note:* Due to experimental conditions, the first snapshot was acquired at a time $t_0$ that might vary between videos. The timestamp information of each frame in each video is available and accessible. The designed model will aim at predicting at specific *times* (and not frame numbers). 

## Setup

### Prerequisites

The following cell will install the required package dependencies, if necessary. You can examine the file, `requirements.txt`, included in the repo to view the list of dependencies.

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
import numpy as np

> **NOTE:** Due to the structure of the challenge, libraries not included in `requirements.txt` will need to be added to be added via a pull request to the [GitHub repo](https://github.com/ramp-kits/bovinsembryos).

Install the `ramp-workflow` package from the master branch on GitHub using the following command in you dedicated python environment.

### The problem.py file

This file contains the definition of the data-challenge according to the RAMP framework. In addition, it contains some useful methods and class.

In [ ]:
from problem import get_train_data, get_test_data, WeightedClassificationError

## Downloading the data

The public data are stored in a public OSF repository, you only need to run the following script which create the `data/` repository locally.

> Note that, in order to be registered to the RAMP studio event, partiticipants must fill the [following form](https://framaforms.org/access-request-to-the-bovmovies2pred-data-challenge-osf-dataset-1646386323) which contain an agreement on the use of the data.

# Data exploration <a name=data_exp></a>

### Loading the data

First, we load the data using the utility function designed for the challenge in `problem.py`. 

In [ ]:
videos_train, labels_train = get_train_data()

`labels_trains` simply consists in the set of labels for the training set, stored as a `numpy.darray`

In [ ]:
print(labels_train[:10])
print(labels_train.shape)
print('Number of videos in the training set: {}'.format(labels_train.size))

On the other hand `videos_train` is a list of objects from the class `VideoReader`. This class is fully described below.

In [ ]:
print(type(videos_train)) # List
print(type(videos_train[0])) # VideoReader object (see below)

### Data visualization 

A video is timelapse of the embryos development, with one image every 15 minutes. One may plot a video image at a specific time (expressed in hour) via the `read_frame` helper method (described below).

In [ ]:
import matplotlib.pyplot as plt
# Here, we use the read_frame method to extract a specific time
# See below for the description of this method
plt.imshow(videos_train[0].read_frame(frame_time=24), cmap='gray')
plt.axis('off')
plt.show()

One may also inspect the class distribution among the 8 possible categories

In [ ]:
labs, counts = np.unique(labels_train, return_counts=True)
plt.bar(labs, counts)
plt.title("Barplot of class distribution in the train dataset")

## The VideoReader class

The `VideoReader` class aims at manipulating the videos. It is basically a wrapper for some openCV features of the library `cv2`, and its main aim is to ease the manipulation of videos for the participants. It is implemented in the `problem.py` file, still we copy its definition here for the sake of self-completeness.

In [ ]:
# You can import the VideoReader class from problem.py via
from problem import VideoReader

In [ ]:
import cv2

class VideoReader:
    def __init__(self, video_filename, frame_times, img_size=[250, 250]):
        self.video = cv2.VideoCapture(video_filename)
        self.nb_frames = int(self.video.get(cv2.CAP_PROP_FRAME_COUNT))
        self.img_size = img_size
        self.frame_times = frame_times

    def read_frame(self, frame_time):
        """Return the frame of a VideoReader object at the specified `frame_time`

        Args:
            frame_time (float): the specified time in hours (allowing quarter hours, e.g. 25.75 or 26.50)

        Raises:
            ValueError: If the specified time does not exist for the selected video

        Returns:
            np.ndarray: A 2-D array containing the grayscale image. 
        """
        if frame_time not in self.frame_times:
            raise ValueError('The specified frame time must me within the time '
                             'interval of the video.')

        frame_nb = np.where(self.frame_times == frame_time)[0][0]
        if frame_nb is not None:
            self.video.set(cv2.CAP_PROP_POS_FRAMES, frame_nb)
        _, frame = self.video.read()

        # always reset video's frame counter to 0 to avoid unexpected behavior
        self.video.set(cv2.CAP_PROP_POS_FRAMES, 0)

        return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    def read_sequence(self, begin_time=None, end_time=None):
        """Extract the sequence of consecutive frames from begin_time to end_time (included)

        Args:
            begin_time (float, optional): The time where the extraction begins. Defaults to None.
            end_time (float, optional):  The time where the extraction ends. Defaults to None.

        Returns:
            np.ndarray: A 3-D numpy array with first axis corresponding to the frame index and the remaining dimension to image size.
        """
        if begin_time is None:
            begin_time = self.frame_times[0]
        elif begin_time not in self.frame_times:
            raise ValueError('The specified begin_time must me within the time'
                             ' interval of the video.')

        if end_time is None:
            end_time = self.frame_times[-1]
        elif end_time not in self.frame_times:
            raise ValueError('The specified end_time must me within the time '
                             'interval of the video.')

        if begin_time > end_time:
            raise ValueError("begin_time must be smaller than end_time.")

        begin_nb = np.where(self.frame_times == begin_time)[0][0]
        end_nb = np.where(self.frame_times == end_time)[0][0]
        self.video.set(cv2.CAP_PROP_POS_FRAMES, begin_nb)

        my_frames = list(range(begin_nb, end_nb + 1))
        video_array = np.empty(
            shape=(len(my_frames), self.img_size[0], self.img_size[1])
        )
        for t, _ in enumerate(my_frames):
            _, frame = self.video.read()
            video_array[t, :, :] = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # always reset video's frame counter to 0 to avoid unexpected behavior 
        self.video.set(cv2.CAP_PROP_POS_FRAMES, 0)

        return video_array

    def read_samples(self, selected_times=None):
        """Read several frames of the video at once corresponding to the selected times.

        Args:
            selected_times (list, optional): The list of of desired extraction times, in hours (allowing quarter hour). Defaults to None, the whole 300 frames are returned.

        Returns:
            np.ndarray: A 3-D numpy array with of shape (size len(selected_times), 250, 250).
        """
        if selected_times is None:
            selected_times = self.frame_times

        res = np.empty([len(selected_times), self.img_size[0], self.img_size[1]])
        frame_nbs = np.where([t in selected_times for t in self.frame_times])[0]
        for i, f in enumerate(frame_nbs):
            self.video.set(cv2.CAP_PROP_POS_FRAMES, f)
            _, frame = self.video.read()

            res[i, :, :] = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        self.video.set(cv2.CAP_PROP_POS_FRAMES, 0)

        return res

### Attributes

The `VideoReader` class has 3 attributes illustrated in the next cell :
- `video`: the video object, which is as cv2.VideoCapture (see [the documentation](https://docs.opencv.org/3.4/d8/dfe/classcv_1_1VideoCapture.html))
- `nb_frames`: the number of frames. This should be 300 for each video.
- `frame_times`:  a 1-D `numpy.array` containing the times at which the frames where recorded. These times are expressed in hours since the fecondation. **All frames** are separated by 15 mns (0.25 hours). However, due to different experimental conditions, videos **might start at different times**!
- `img_size` returns the dimensions of the square images composing the video. This should be normalized to $250 \times 250$.



In [ ]:
example_video = videos_train[0]
print("A video is stored as a " + str(type(example_video.video)) + " Python object.")
print("It has " + str(example_video.nb_frames) + " frames of size " + str(example_video.img_size) + ".")
print("The frame are recorded at the following times (expressed as time in hour since the fecondation)")
print(example_video.frame_times[0:30])

### Methods
The `VideoReader` class has methods illustrated in the next cell :
- The `read_frame` method returns the frame of the video at a specified time.  The user must provide the `frame_time` argument, which is a time present in the `frame_times` attribute. This method returns a 2-D `numpy.ndarray`.
- The `read_sequence` method returns a sequence of **consecutive frames**. The user must provide the `begin_time` argument (the frame time at which the sequence begins) and the `end_time` (the frame time at which the sequence ends). By default, these two arguments are set to `None`, which results in starting at first time and ending at last time. The method returns a 3-D ($n \times 250 \times 250$) `numpy.ndarray`, where $n$ is the number of consecutive times between `begin_time` and `end_time` (included).
- The `read_samples` method transforms the video to a 3-D ($300 \times 250 \times 250$) `numpy.ndarray`. Optionally, a `selected_times` argument (provided as a list) may be specified to only select frames corresponding to these times.

In [ ]:
selected_times = example_video.frame_times[[0, 200, 299]] 
print("Plot the videos frames corresponding to times: " + str(selected_times) + " hours.")
for frame_time in selected_times:
    plt.imshow(example_video.read_frame(frame_time), cmap="gray")
    plt.axis('off')
    plt.show()

In [ ]:
video_array = example_video.read_sequence()
video_array.shape

In [ ]:
video_array = example_video.read_sequence(end_time = 29)
print(video_array.shape)

video_array = example_video.read_sequence(begin_time = 28.75, end_time=29)
print(video_array.shape)

In [ ]:
video_array = example_video.read_samples(selected_times=[23.75, 62.25, 96])
video_array.shape

> **Caution:** the VideoReader class does not load all the videos frame into memory, which is useful in order not to overload you local machine or the RAMP servers. However, the `read_sample` or `read_sequence` may quickly lead to memory issues when trying to load every videos. You code should take this issue into account and treat videos sequentially or, at least, batch-wise.

# Designing a submission

### Evaluation

For the moment, we only consider the overall classification accuracy of your classifiers.

Locally, the RAMP platform use a 3-fold cross-validation scheme implemented for you in the `get_cv` method. Moreover, the classifier's performance is evaluated on a separate test set which can be loaded just as the training data. 

In [ ]:
videos_test, labels_test  = get_test_data()

In [ ]:
print('Number of subjects in the test set: {}'.format(labels_test.size))

### Handling several prediction times

Remember that the challenge's objective is to design classifier(s) for different prediction times between 1 and 4 days (24 to 96 hours). Your submission should account for that constraints with a `pred_time` argument for the model fit and prediction, ensuring that only frames up to `pred_times` are used for model training and testing.

The predefined and fixed prediction times are

In [ ]:
pred_times = [27, 32, 37, 40, 44, 48, 53, 58, 63, 94, None]


> **Important:** You are freed to opt for having one model for each prediction time, or one model for every time. However, note that for the second option your model will be fully retrained at each prediction time, and that it must handle videos with a varying number of frames.

### Mandatory structure of a submission


A submission (usually stored in `./subsmissions/<submission_foldername>/`) must contain on file named `videoclassifier.py`.

This python script must itself implement at least a `VideoClassifier` class with
 * A `fit(videos, y, pred_time)` method.
 * A `pred(videos, pred_time)` method.

The two arguments must be understood as follow:
 * `videos` is a list of `VideoReader` object with videos object cut at `pred_times`.
 * `y` is a 1-D numpy array containing the associated training labels.
 * `pred_time` is the time of development for which the prediction shall be made. 

We illustrate this below with a simple example.




### Illustration : a dummy random classifier

This classifier does not use (nor even load into memory) the videos or the prediction time, and just predict random labels. Still, it is a valid (albeit unuseful) submission regarding the RAMP workflow.

It is implemented in the `./subsmissions/starting_kit/` folder and we copy its code here for illustration.

In [ ]:
import numpy as np


class VideoClassifier(object):
    def __init__(self):
        pass

    def fit(self, videos: list, y, pred_time: float):
        classes = ["A", "B", "C", "D", "E", "F", "G", "H"]
        self.n_classes = len(classes)
        pass

    def predict(self, videos: list, pred_time: float):
        proba = np.random.rand(len(videos), self.n_classes)
        proba /= proba.sum(axis=1)[:, np.newaxis]
        return proba

In [ ]:
my_pred_time = 96
my_model = VideoClassifier()
my_model.fit(videos_train, labels_train, pred_time=my_pred_time)
y_pred = my_model.predict(videos_test, pred_time=my_pred_time)

#### Score function

For now, only weighted classification error is implemented. The LOWER the better. This score is based on weighted errors. The weights were determined by an expert.

In [ ]:
def lett_to_num(y):
    """Return a numeric encoding of the letters"""
    return [ord(lab) - 65 for lab in y]

In [ ]:
wce = WeightedClassificationError(time_idx=my_pred_time)
wce.compute(y_true=lett_to_num(labels_test), y_pred=y_pred)

In [ ]:
for my_pred_time in pred_times:
    my_model = VideoClassifier()
    my_model.fit(videos_train, labels_train, pred_time=my_pred_time)
    y_pred = my_model.predict(videos_test, pred_time=my_pred_time)
    wce = WeightedClassificationError(time_idx=my_pred_time)
    print('Classification error at time ' + str(my_pred_time) + ' is:', str(wce.compute(y_true=lett_to_num(labels_test), y_pred=y_pred)))

To compare to the performance of a (dummy) classifier, returning only one of the 8 classes

In [ ]:
for cl in np.unique(labels_train):
    y_cl = np.zeros((labels_train.size, np.unique(labels_train).size))
    y_cl[:, lett_to_num(cl)] = 1
    print('Classifying all videos in class', cl, 'gives a score of:', wce.compute(lett_to_num(labels_train), y_cl))

### Submitting to RAMP

Before submitting to RAMP, you can test your solution locally to ensure that trivial errors (e.g. typos, path issues, etc.) are resolved. We can test a given submission using the `ramp` command that was installed in the virtual environment.  
We'll use the following command:  
`!ramp test --submission <subm_folder> --quick-test`  
The `!` signals that the command should be run on the command line instead of this notebook.  
`ramp-test` is the command to be executed. It signals `ramp` to perform a local test. 
`--submission <subm_folder>` specifies which submission to run. You can have multiple potential submissions in the `submissions/` directory; this prevents `ramp` from running all of them.  

In [ ]:
!ramp-test --quick-test --submission starting_kit

We can see that the results are not very good, but that is expected: our estimator in starting_kit completely ignores the data and returns a random prediction!

RAMP will automatically perform 3-fold cross-validation and report the WeightedClassificationError for each of the folds and each prediction times, along with the mean across the folds. Bagging of the results has been disabled; the output can be ignored.

## More information

You can find more information in the [README](https://github.com/paris-saclay-cds/ramp-workflow/blob/master/README.md) of the [ramp-workflow library](https://github.com/paris-saclay-cds/ramp-workflow).